In [1]:
import cv2
import pandas as pd
import numpy as np

def preprocessing(val=True):
    
    data = pd.read_csv('data/x_train_gr_smpl.csv')
    labels = pd.read_csv('data/y_train_smpl.csv')

    if(val):
        ## ---------------- Data preparation ---------------- ##
        X_train = []
        for i in range(data.shape[0]):
            img = np.uint8(data.iloc[i])
            edited = cv2.Canny(img, 10, 30)
            edited = cv2.GaussianBlur(edited, (5, 5), 0)
            X_train.append(edited.reshape((1,-1))[0])

        data = pd.DataFrame(X_train)
        ## -------------------------------------------------- ##

    return data, labels

In [2]:
from sklearn.model_selection import train_test_split

data, labels = preprocessing()
data['label'] = labels

X = data.iloc[:, :-1]
y = data['label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)

In [3]:
from sklearn.neural_network import MLPClassifier

In [4]:
clf = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(5, 2), random_state=1)
clf.fit(X_train, y_train)

MLPClassifier(activation='relu', alpha=1e-05, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(5, 2), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
       random_state=1, shuffle=True, solver='lbfgs', tol=0.0001,
       validation_fraction=0.1, verbose=False, warm_start=False)

In [43]:
y_pred = []
for x in X_test:
    y_pred.append(clf.predict_proba(x))

ValueError: Expected 2D array, got scalar array instead:
array=0.
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.

In [32]:
print(y_pred[:10])

[[0.10940128 0.14306252 0.03296664 0.10180573 0.16650347 0.17498093
  0.06333357 0.0201197  0.16358401 0.02424214]
 [0.10940128 0.14306252 0.03296664 0.10180573 0.16650347 0.17498093
  0.06333357 0.0201197  0.16358401 0.02424214]
 [0.10940128 0.14306252 0.03296664 0.10180573 0.16650347 0.17498093
  0.06333357 0.0201197  0.16358401 0.02424214]
 [0.10940128 0.14306252 0.03296664 0.10180573 0.16650347 0.17498093
  0.06333357 0.0201197  0.16358401 0.02424214]
 [0.10940128 0.14306252 0.03296664 0.10180573 0.16650347 0.17498093
  0.06333357 0.0201197  0.16358401 0.02424214]
 [0.10940128 0.14306252 0.03296664 0.10180573 0.16650347 0.17498093
  0.06333357 0.0201197  0.16358401 0.02424214]
 [0.10940128 0.14306252 0.03296664 0.10180573 0.16650347 0.17498093
  0.06333357 0.0201197  0.16358401 0.02424214]
 [0.10940128 0.14306252 0.03296664 0.10180573 0.16650347 0.17498093
  0.06333357 0.0201197  0.16358401 0.02424214]
 [0.10940128 0.14306252 0.03296664 0.10180573 0.16650347 0.17498093
  0.06333357

In [29]:
correct = 0
incorrect = 0
y_t = []

for y in y_test:
    y_t.append(y)
for i in range(len(y_pred)):
    if y_pred[i] == y_t[i]:
        correct += 1
    elif y_pred[i] != y_t[i]:
        incorrect += 1

acc = correct / (correct + incorrect)

print(acc)

0.16419339396840593
